In [ ]:
import mne
import plotly.graph_objects as go
import numpy as np

# Путь к файлу
file_path = 'data/train_dataset_minzdrav_train/ECoG_fully_marked_(4+2 files, 6 h each)/Ati4x3_12m_BL_6h_fully_marked.edf'
# Какие сегменты и индексы будем отображать (отображение идёт в отсортированном порядке)
available_segments = ['swd', 'is', 'ds']
indices_to_display = (0, 2)

# Загрузка EDF файла
raw = mne.io.read_raw_edf(file_path, preload=True)

# Извлечение данных в numpy массив
data = raw.get_data()
times = raw.times

# Извлечение аннотаций
annotations = raw.annotations

# Создаем словарь для хранения отрезков по типам
segments = {
    'swd1': [], 'swd2': [], 
    'is1': [], 'is2': [],
    'ds1': [], 'ds2': []
}

# Собираем все отрезки по типам
for annotation in annotations:
    onset = int(annotation['onset'] * raw.info['sfreq'])
    description = annotation['description']
    if description in segments:
        segments[description].append(onset)

# Находим последовательные сегменты
consecutive_segments = []
for segment_type in available_segments:
    for i in range(len(segments[segment_type + '1'])):
        consecutive_segments.append((segment_type, i, segments[segment_type + '1'][i]))

# Сортируем по индексу
consecutive_segments.sort(key=lambda x: x[2])

for segment_type, index, _ in consecutive_segments[indices_to_display[0]:indices_to_display[1]]:
    target_onset = segments[segment_type + '1'][index]
    target_duration = segments[segment_type + '2'][index] - target_onset
    
    window_before = int(target_duration)
    window_after = int(target_duration)
    
    start_idx = max(0, target_onset - window_before)
    end_idx = min(len(times), target_onset + target_duration + window_after)
    
    # Создание фигуры с одним графиком
    fig = go.Figure()

    # Определение цветов для каждого типа маркера
    label_colors = {
        'swd1': 'red',
        'swd2': 'red', 
        'is1': 'green',
        'is2': 'green',
        'ds1': 'orange',
        'ds2': 'orange'
    }

    # Добавление всех трех каналов на один график
    fig.add_trace(go.Scatter(x=times[start_idx:end_idx], y=data[0][start_idx:end_idx], name='Канал 1'))
    fig.add_trace(go.Scatter(x=times[start_idx:end_idx], y=data[1][start_idx:end_idx], name='Канал 2'))
    fig.add_trace(go.Scatter(x=times[start_idx:end_idx], y=data[2][start_idx:end_idx], name='Канал 3'))

    # Добавляем вертикальные линии для обозначения начала и конца события
    fig.add_vline(x=times[target_onset], line_dash="dash", line_color=label_colors[segment_type + '1'], annotation_text=segment_type + '1')
    fig.add_vline(x=times[target_onset + target_duration], line_dash="dash", line_color=label_colors[segment_type + '2'], annotation_text=segment_type + '2')

    # Обновление макета
    fig.update_layout(
        title_text=f"ЭКоГ данные - окно вокруг первого {segment_type} события",
        showlegend=True,
        height=500,
        width=1000,
        xaxis_title='Время (с)',
        yaxis_title='Амплитуда'
    )

    fig.show()